In [ ]:
import pandas as pd
import requests as rqst
from bs4 import BeautifulSoup

Define useragent and url

In [ ]:
baseurl='https://www.etsy.com/'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

# Set the headers to include the user agent
headers = {'User-Agent': user_agent}

In [ ]:
r = rqst.get('https://www.etsy.com/market/budget_planner')
soup = BeautifulSoup(r.content,'lxml')

productlist= soup.find('ul',{'id':['search-results']})


In [ ]:
# Find all the links to the products
product_links = []
for link in soup.find_all('a', class_='listing-link', href=True):
    product_links.append(link['href'])